# Import Modules

In [1]:
import os
import sys

from os import listdir
from pathlib import Path

import numpy as np
import pandas as pd

# from tqdm import tqdm
from tqdm.notebook import tqdm

# #########################################################
from ase import io

# #########################################################
from misc_modules.misc_methods import GetUniqueFriendlyID

In [2]:
import chart_studio.plotly as py
import plotly.graph_objs as go

In [3]:
root_dir = os.path.join(
    os.environ["gdrive"],
    "norskov_research_storage/nersc/IrOx_Project_temp_190510")
outcar_locations = []
for subdir, dirs, files in os.walk(root_dir):
    if "OUTCAR" in files:
        # print(subdir)
        outcar_locations.append(subdir)

print("Number of locations with an OUTCAR file:", "\n", len(outcar_locations))

Number of locations with an OUTCAR file: 
 879


In [5]:
subdir

'/mnt/f/GDrive/norskov_research_storage/nersc/IrOx_Project_temp_190510/__temp__/1-att/_3'

In [4]:
assert False

AssertionError: 

In [ ]:
# path_i = "/mnt/f/GDrive/norskov_research_storage/nersc/IrOx_Project_temp_190510/01_surface_calcs/IrO2/100/01_layers/_1"
# path_i = "/mnt/f/GDrive/norskov_research_storage/nersc/IrOx_Project_temp_190510/01_surface_calcs/IrO2/100/03_layers/2-temp"

def include_path(path_i):
    """
    """
    # #########################################################
    last_dir = path_i.split("/")[-1]

    start_with_ = False
    if last_dir[0] == "_":
        start_with_ = True

    bool_list = []
    for i in last_dir[1:]:
        bool_list.append(i.isnumeric())
    all_numeric = all(bool_list)

    correct_format = False
    if start_with_ and all_numeric:
        correct_format = True

    # #########################################################
    blacklist_folders = ["__temp__", "__failed__"]
    blacklist_path = False
    for blacklist_i in blacklist_folders:
        if blacklist_i in path_i:
            blacklist_path = True


    # #########################################################

    include_path = False
    if correct_format and not blacklist_path:
        include_path = True
    
    return(include_path)

def parse_cpu_time(outcar_lines):
    # outcar_list = row_i.outcar
    outcar_list = outcar_lines
    search_lines = [i for i in outcar_list if "Total CPU time used" in i]
    if len(search_lines) == 1:
        time_i = float(search_lines[0].split()[-1])
        return(time_i)    
    else:
        return(None)

def get_prepath_rev_att(path_i):
    # #########################################################
    data_dict_i = dict()

    # #########################################################
    pre_path = "/".join(path_i.split("/")[0:-1])
    data_dict_i["pre_path"] = pre_path

    # #########################################################
    last_dir = path_i.split("/")[-1]
    rev_i = int(last_dir[1:])
    data_dict_i["rev"] = rev_i

    # #########################################################
    dir_i = path_i.split("/")[-2]
    if "_attempt" in dir_i:
        att_i = int(dir_i.split("_")[0])
    else:
        att_i = None
    data_dict_i["attempt"] = att_i

    return(data_dict_i)

def scf_cycles_model(slope, intercept, num_atoms=None):
    y = slope * num_atoms + intercept

    print("y:", y)

    out = 10 ** (y)
    return(out)

In [ ]:
# TEMP
# outcar_locations = outcar_locations[0:100]

In [ ]:
data_dict_list = []
for dir_i in outcar_locations:
    data_dict_i = dict()

    data_dict_i["path"] = dir_i

    # #####################################################
    include_path_i = include_path(dir_i)
    data_dict_i["include"] = include_path_i

    if include_path_i:
        out_dict = get_prepath_rev_att(dir_i)

        data_dict_i["attempt"] = out_dict["attempt"]
        data_dict_i["rev"] = out_dict["rev"]
        data_dict_i["pre_path"] = out_dict["pre_path"]

    # #####################################################
    is_vib_calc = False
    for file_i in listdir(dir_i):
        if ".eq.pckl" in file_i:
            is_vib_calc = True

    data_dict_i["vib_calc"] = is_vib_calc
    
    # #####################################################
    data_dict_list.append(data_dict_i)

df_vib = pd.DataFrame(data_dict_list)

# df_vib[df_vib.vib_calc == True]

In [ ]:
# data_dict_list = []
# # for dir_i in outcar_locations:
# iterator = tqdm(outcar_locations, desc="1st loop")
# for i_cnt, dir_i in enumerate(iterator):
#     tmp = 42

#     data_dict_i = dict()
#     # print("dir_i:", dir_i)
#     print(dir_i)

#     data_dict_i["path"] = dir_i

#     # #####################################################
#     num_images = None
#     try:
#         traj = io.read(
#             os.path.join(dir_i, "OUTCAR"),
#             index=":")
#         num_images = len(traj)
#     except:
#         pass

#     data_dict_i["num_images"] = num_images
            
#     # #####################################################
#     for file_i in listdir(path=dir_i):


#         if "init" in file_i:
            
#             atoms_i = io.read(
#                 os.path.join(
#                     dir_i,
#                     file_i))

#             # atoms_i = traj[-1]
#             num_atoms = atoms_i.get_number_of_atoms()
#             data_dict_i["num_atoms"] = num_atoms




#             break

#     # #####################################################
#     path_i = os.path.join(dir_i, "OUTCAR")
#     my_file = Path(path_i)
#     if my_file.is_file():
#         with open(path_i, "r") as f:
#             outcar_lines = f.read().splitlines()

#         num_lines = len(outcar_lines)

#         cpu_time_i = parse_cpu_time(outcar_lines)

#         data_dict_i["time"] = cpu_time_i
#         data_dict_i["num_OUTCAR_lines"] = num_lines
#         data_dict_list.append(data_dict_i)



# # #########################################################
# df = pd.DataFrame(data_dict_list)

# ids_list = []
# for i in range(df.shape[0]):
#     id_i = GetUniqueFriendlyID(ids_list)
#     ids_list.append(id_i)

# df["ids"] = ids_list
# df = df.set_index("ids")

# df["time_min"] = df.time / 60

# print("Number of rows", df.shape[0])

# # Pickling data ###########################################
# import os; import pickle
# directory = "out_data"
# if not os.path.exists(directory): os.makedirs(directory)
# with open(os.path.join(directory, "df_outcar.pickle"), "wb") as fle:
#     pickle.dump(df, fle)
# # #########################################################

In [ ]:
# #########################################################
import pickle; import os
path_i = os.path.join("out_data", "df_outcar.pickle")
with open(path_i, "rb") as fle:
    df = pickle.load(fle)
# #########################################################

In [ ]:
df["images_per_min"] = df.num_images / df.time_min

df["images_per_min_log"] = np.log(df.images_per_min)

df = pd.concat([
    df.reset_index().set_index("path"),
    df_vib.reset_index().set_index("path"),
    ], axis=1)

df = df.reset_index()
df = df.set_index("ids")

In [ ]:
df_i = df

df_i = df_i[df_i.vib_calc == False]
df_i = df_i[df_i.include == True]
df_i = df_i[~df_i.time.isnull()]


# df_i

In [ ]:
data = []

# Log-linear model

In [ ]:
from sklearn.linear_model import LinearRegression

y = df_i.images_per_min_log.tolist()

X = df_i.num_atoms.to_numpy()
X = X.reshape(-1, 1)

reg = LinearRegression().fit(X, y)
reg.score(X, y)

print("reg.coef_:", reg.coef_)
print("reg.intercept_:", reg.intercept_)

x_array = np.linspace(0, 100, num=101).reshape(-1, 1)
y_array = [10 ** (i) for i in reg.predict(x_array)]


# #########################################################
trace = go.Scatter(
    x=x_array.flatten(),
    y=y_array,
    # text=df.index,
    name="log-linear model",
    mode="lines",
    )
data.append(trace)

# Polynomial model

In [ ]:
def scf_poly_model(num_atoms, m0=None, m1=None, m2=None):


    # X_min_ = poly.fit_transform(num_atoms)
    # scf_per_min_i = lg.predict(X_min_)[0]

    # print("m0:", m0, "m1:", m1, "m2:", m2)

    # #####################################################
    scf_per_min_i = 0 + \
        (m0) * (num_atoms ** 0) + \
        (m1) * (num_atoms ** 1) + \
        (m2) * (num_atoms ** 2)
    return(scf_per_min_i)

def scf_per_min_model(num_atoms=None):
    # m0 = lg.intercept_
    # m1 = lg.coef_[1]
    # m2 = lg.coef_[2]

    m0 = +6.47968662
    m1 = -0.12968329
    m2 = +0.00068543


    # min_x = -(m1 / (2 * m2))
    x_min = -(m1 / (2 * m2))

    X_min = np.array([x_min]).reshape(-1, 1)

    y_min = scf_poly_model(x_min, m0=m0, m1=m1, m2=m2)

    
    # Linear region paramters
    max_lin_x = 200  # End of linear region
    x0 = x_min
    y0 = y_min
    x1 = max_lin_x
    y1 = 0.06
    if num_atoms >= x_min and num_atoms <= max_lin_x:
        m = (y1 - y0) / (x1 - x0)
        b = (x1 * y0 - x0 * y1) / (x1 - x0)

        scf_per_min_i = m * num_atoms + b

        # print("min_y:", min_y)
        # scf_per_min_i = y_min / 2

    elif num_atoms > max_lin_x:
        scf_per_min_i = y1

    else:
        scf_per_min_i = scf_poly_model(num_atoms, m0=m0, m1=m1, m2=m2)

        # X_test = np.array([num_atoms, ]).reshape(-1, 1)
        # X_test_ = poly.fit_transform(X_test)
        # scf_per_min_i = lg.predict(X_test_)[0]

    return(scf_per_min_i)

def calc_wall_time(num_atoms=None, num_scf=None):
    scf_per_min_i = scf_per_min_model(num_atoms=num_atoms)
    wall_time_i = num_scf / scf_per_min_i

    return(wall_time_i)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

y = df_i.images_per_min.tolist()

# X = df_i.num_atoms.to_numpy()
# X = X.reshape(-1, 1)

# # PolynomialFeatures (prepreprocessing)
# poly = PolynomialFeatures(degree=2)
# X_ = poly.fit_transform(X)

# lg = LinearRegression()
# lg.fit(X_, y)


x_array = np.linspace(0, 250, num=901)
y_array = [scf_per_min_model(num_atoms=i) for i in x_array]


#########################################################
trace = go.Scatter(
    # x=x_array.flatten(),
    x=x_array,
    y=y_array,
    # text=df.index,
    name="Polynomial",
    mode="lines",
    )
data.append(trace)

In [ ]:
# print(lg.coef_)
# print(lg.intercept_)

num_atoms = 100
(+6.47968662) * (num_atoms ** 0) + (-0.12968329) * (num_atoms ** 1) + (+0.00068543) * (num_atoms ** 2)

In [ ]:

# Obtain coefficients
# lg.coef_
# y_array = lg.predict(X_test_)

In [ ]:
# assert False

# Plotting

In [ ]:
df.head()

In [ ]:
trace = go.Scatter(
    x=df.num_atoms,

#     y=df.time_min,
    y=df.images_per_min,
    # y=df_i.images_per_min_log,

    text=df.index,
    mode="markers",
    marker=dict(
        opacity=0.8,
        size=14,
        ),
    )
data.append(trace)

layout = go.Layout(
    xaxis=dict(title=dict(text="Num Atoms")),
    yaxis=dict(
        title=dict(text="SCF Cycles/min"),
        # type="log",
        ),
    )

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# 200

1/200 * 60

In [ ]:
from plotting.my_plotly import my_plotly_plot

my_plotly_plot(
    figure=fig,
    plot_name="scf_per_min_vs_atoms",
    write_html=True,
    write_png=False,
    png_scale=6.0,
    write_pdf=False,
    write_svg=False,
    try_orca_write=False,
    )

In [ ]:
data = []

desired_scf_cycles = 100

y_array = []
x_array = np.linspace(0, 200, num=401)
for num_atoms_i in x_array:
    # scf_per_min_i = scf_per_min_model(num_atoms=num_atoms_i)
    # wall_time_i = desired_scf_cycles / scf_per_min_i
    wall_time_i = calc_wall_time(num_atoms=num_atoms_i, num_scf=100)

    y_array.append(wall_time_i / 60.)

# #########################################################
trace = go.Scatter(
    x=x_array,
    y=y_array,

    # text=df.index,
    # mode="markers",
    # marker=dict(
    #     opacity=0.8,
    #     size=14,
    #     ),
    )
data.append(trace)

layout = go.Layout(
    xaxis=dict(title=dict(text="Num Atoms")),
    yaxis=dict(
        title=dict(text="Run time (hr)"),
        # type="log",
        ),
    )

fig = go.Figure(data=data, layout=layout)
fig.show()

# Grouping by XX_attempt and _X revisions

In [ ]:
df_i

grouped = df_i.groupby(["pre_path"])
for name, group in grouped:
    tmp = 42

    # print(group.shape[0])

# group.iloc[0].pre_path
group.iloc[0].path

In [ ]:
# df_i[df_i.path == "/mnt/f/GDrive/norskov_research_storage/nersc/IrOx_Project_temp_190510/07_diff_coverages_term/IrO3_rutile-like/110/bare_covered/_3"]

df[df.path == "/mnt/f/GDrive/norskov_research_storage/nersc/IrOx_Project_temp_190510/07_diff_coverages_term/IrO3_rutile-like/110/bare_covered/_3"]

In [ ]:
assert False

In [ ]:
# phrase_i = "muteg"
phrase_i = "sihe"

matching_ids = [i for i in df.index if phrase_i in i]

matching_ids

In [ ]:
row_i = df.loc[matching_ids[0]]

print(row_i.path, "\n")

row_i

In [ ]:
# for file_i in listdir(path=dir_i):
#     if "init" in file_i:
#         print(file_i)
#         atoms_i = io.read(
#             os.path.join(
#                 dir_i,
#                 file_i)
#             )
#         num_atoms = atoms_i.get_number_of_atoms()
#         break



In [ ]:
# # path_i = "/mnt/f/GDrive/norskov_research_storage/nersc/IrOx_Project_temp_190510/01_surface_calcs/IrO3/100/41_layers/_1"
# path_i = "/mnt/f/GDrive/norskov_research_storage/nersc/IrOx_Project_temp_190510/03_OER_Calc/IrO3_battery/010/01_surface_type_a/01_O_covered/01_bare/01_attempt/_1"

# # #########################################################
# data_dict_i = dict()

# # #########################################################
# pre_path = "/".join(path_i.split("/")[0:-1])
# data_dict_i["rev"] = rev_i

# # #########################################################
# last_dir = path_i.split("/")[-1]
# rev_i = int(last_dir[1:])

# # #########################################################
# dir_i = path_i.split("/")[-2]
# if "_attempt" in dir_i:
#     att_i = int(dir_i.split("_")[0])
# else:
#     att_i = None
# data_dict_i["attempt"] = att_i

# # #########################################################
# data_dict_list.append(data_dict_i)

# # pd.DataFrame(data_dict_list)



In [ ]:
# data_dict_list = []
# # for path_i in df_i.path.tolist():
# for path_i in outcar_locations:

#     # #########################################################
#     data_dict_i = dict()

#     # #########################################################
#     pre_path = "/".join(path_i.split("/")[0:-1])
#     data_dict_i["pre_path"] = pre_path

#     # #########################################################
#     last_dir = path_i.split("/")[-1]
#     rev_i = int(last_dir[1:])
#     data_dict_i["rev"] = rev_i

#     # #########################################################
#     dir_i = path_i.split("/")[-2]
#     if "_attempt" in dir_i:
#         att_i = int(dir_i.split("_")[0])
#     else:
#         att_i = None
#     data_dict_i["attempt"] = att_i

#     # #########################################################
#     data_dict_list.append(data_dict_i)


# df_paths = pd.DataFrame(data_dict_list)

# df_i.reset_index().set_index("path")